## CREATE EPOCHS AND LABEL DATA BASED ON DA SYMBOL STIMULI
The following file create annotations based on divided attention(DA) timepoints as they appear in the video.
this enables the epochs data to be seperated into right and left visual field 
epochs are then created


In [ ]:
import matplotlib 
import pathlib
import re
import os
import mne
from mne.datasets.eyelink import data_path
from mne.preprocessing.eyetracking import read_eyelink_calibration
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from mne.time_frequency import tfr_morlet

In [ ]:
subject='11766'# '15463'#
subj='1'  #'8'#
task='DA'  #'HD'

path1=r'C:\Users\lpxaj7\OneDrive - The University of Nottingham\Documents\Experiments\Year_1\v0_1\\'
path1= os.path.join(path1,task)

savefigpath= os.path.join(path1,'analysis',subject,'button_source','')
os.makedirs(savefigpath, exist_ok=True)


#MEG
filename_meg=str(subj+'_'+task+'_3_raw_ica_hfc_meg.fif')
if subject=='15463':
    filename_meg=str(subj+'_'+'raw_ica_hfc_meg.fif')

read_file_meg=os.path.join(path1,'all_files',subject,filename_meg)

# #ET
savefigpath1=r'C:\Users\lpxaj7\OneDrive - The University of Nottingham\Documents\codes\eyetracker\\'
fig_path_source= os.path.join(path1,'analysis','sourcespace')

In [ ]:
participant=subject

In [ ]:
##%% load data

#MEG
raw_meg=mne.io.read_raw_fif(read_file_meg,preload=True)
print(raw_meg.info)

# also check what time is mentioned as first time of the samples
raw_meg.copy()._first_time # that is the origin or starting time

original_raw_meg=raw_meg.copy()

In [ ]:
##%% visualise raw data
mag_channels= mne.pick_types(raw_meg.info,meg='mag')
scalings = {'mag':1e-11,'stim':3}


#raw_meg.drop_channels(['et_x'])
  # plot all channels together with a duration of 60 secs
raw_meg.plot(title='RAW DATA', n_channels=len(mag_channels),duration=60)

##%% channels marked as bad check
print(raw_meg.info["bads"])

In [ ]:
##%% broadband filter and psd
fmin=1
fmax=99
# tmin= 
raw_bb= raw_meg.filter(fmin,fmax)
raw_bb.plot_psd(fmin=0, fmax=fmax+10,  exclude='bads')#,title=f' PSD {subject} {fmin}_{fmax}')
plt.show()
##%% apply hfc
raw= raw_bb.copy()
# raw_bb2= raw_bb.copy()

bad_ch=['T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11',\
        'A1','A3','A5','A7','A8','A10','A12','A14','A16']
raw_bb.info['bads'].extend(bad_ch)

In [ ]:
#read files
##%%et
#ET

filename_et= (subj+'_'+task+'.asc')
read_file_et=os.path.join(path1,os.pardir,os.pardir,'v0\\DrivingSim\\ET\\ET_allfiles',filename_et)

raw_et = mne.io.read_raw_eyelink(read_file_et)#, preload=True)#,create_annotations=["blinks"])

raw_et.info
##%%
#raw_events=raw_bb.copy()
##%% da_time

#Divided attention timepoints file 
da_time_file= savefigpath1 + f'da_time_{subj}.csv'   # all data of da symbols appear from eyetracker video
da_time= pd.read_csv(da_time_file,header=None,names=['Time'])

In [ ]:
##%% selecting events
# select events based on button press only for DA task ehich is between 200 to 700ms
raw_file=raw_beta.copy()

#raw_file=raw.copy()
#del raw
events= mne.find_events(raw_file.copy().crop(tmin=200, tmax=700), stim_channel="button press")
raw_file.copy().plot(events=events)


# for epoch data just filter da events for this analysis

raw_file_epochs= raw_file.copy().crop(tmin=200, tmax=700).pick_types(meg=True, stim=False)




##%%
#raw_file.annotations.rename({'saccade':'BAD_saccade'})

In [ ]:
##%% labelling events based on DA symbol appeareance (32)

old_annot= raw_file.annotations.onset - raw_file._first_time # (subtracting this to accomodate for first sample automatic correction  by mne)
old_annot_descp= raw_file.annotations.description
old_annot_duration= raw_file.annotations.duration

# the screen recording of et started from this point w and we have obtained symbol values from this recording therefore need to add this value to all symbol appearance data
drive_strart = raw_et.annotations[raw_et.annotations.description == '-6 Record Screen Started: screenrecording.mp4']
dri_start_time= drive_strart.onset  #raw_et._first_time=0

#conver df to 1d numpy array
# onset_da=da_time.values.flatten()+drive_strart.onset# +raw_file._first_time  # raw_file.annotations.onset[-1]
onset_da=da_time+drive_strart.onset# +raw_file._first_time  # raw_file.annotations.onset[-1]

 # FIRST LETTER REPRESENt symbol direction, 2nd represent visual field in which presented
decp=['LR', 'LL', 'UL', 'LL',\
'DL', 'LR', 'RR', 'UR',\
'RR', 'UL', 'DR', 'RL',\
'LR', 'DR', 'DL', 'RR',\
'UR', 'LL', 'LR', 'DL',\
'RL', 'RL', 'UR', 'DR',\
'RL', 'DR', 'UL', 'RR',\
'LL', 'UL', 'DL', 'UR']
description_da= decp   #['DA']*len(onset_da)
dur_da= [4.5]*len(onset_da)

onset=[]; description=[];duration=[]
onset= np.append(old_annot, onset_da)
#onset= onset * (500 / 375)

description= np.append(old_annot_descp,description_da)

duration=np.append(old_annot_duration,dur_da)#[0]*len(onset)

annotations = mne.Annotations(onset, duration=duration, description=description)#, orig_time=raw_events._first_time)
#raw_file.set_annotations(annotations)

# Add annotations to raw data
raw_events=raw_file.copy()

raw_events.set_annotations(annotations)
raw_events.plot()
#raw_events.copy().pick_channels(["button press"]).plot(events=events,picks='stim')
#del raw_events

In [ ]:
##%% relabel events based on annotations
annot_dict= {0: ['UL', 'LR', 'RL', 'DR'], 
 1: ['DL', 'LL'], 
 4: ['RR', 'UR']}

labelled_event=[]
for event in events:
    # Get the time of the event in seconds
    event_time = round((event[0] / raw_file.info['sfreq']-raw_file._first_time),2)  # remove first sample time
    event= np. insert(event, 3, event_time)

    print(event_time)
     # Find the most recent annotation before the event  
    for ann in annotations:
        if (ann['description'] in decp and (ann['onset']<= event_time) & (ann['onset']+10 >= (event_time))):  # 10 is added just to ensurethat button might have been pressed even after 5 seconds of symbol disaapearence (symbol diaapears after 4.5 sec: 4.5+5 )
            print(ann)    
            prev_annotations= ann            
            descp= int([key for key, value in annot_dict.items() if prev_annotations['description'] in value][0])
            diff= event_time-ann['onset'] # tells difference between button press and symbol time
            print('ons= ',ann['onset'] ) 
            
            
     
    # prev_annotations = [ann for ann in annotations if ann['onset'] < event_time]
    # #print(prev_annotations)
    # Get the label of the most recent annotation if there is any
    if prev_annotations:
        event[2]= descp
        event= np.insert(event,4, diff)
        #event=np.insert(event,5, onsett)
    else: 
        continue
    # Append the event and its label to the labeled events list
    labelled_event.append(event)

# convert the list to an array
event_list=np.array([row[:3] for row in labelled_event])  # keep only first 3 columns tthat contain event details

#added line on 4/11/24
raw_file_epochs= raw_file.copy().crop(tmin=200, tmax=700).pick_types(meg=True, stim=False)

In [ ]:
##%%% create epochs
event_dict= {
    'leftVF':1,
    'rightVF':4
    #'incorrect':0
    }

##%%%%%%%


epochs_new= mne.Epochs(
    raw_file_epochs, 
    event_list, 
    tmin= -2, 
    tmax=2, 
    event_id= event_dict, 
    reject_by_annotation=False, 
    picks=['meg'],
    baseline=None
 )

epochs_new.plot(n_epochs=10, events=True)
##%% removing bad epochs thru sd

In [ ]:
#clean epochs; 
epoch_data= epochs_new.get_data()   #array of shape (n_epochs, n_channels, n_times)
epochs_sd= epoch_data.std(axis=2)    # Standard deviation across time within each epoch for each channel
  #average standard deviation of the signal at that sensor across all trials

time_sd= epoch_data.std(axis=0)   # Standard deviation across epochs  for each channel x time
mean_time_sd= time_sd.mean(axis=1)  # mean of above

# Get the channel names from the info
channel_names = np.array(epochs_new.ch_names)

# Dictionary to store bad channels for each epoch
bad_channels_per_epoch = {}
bad_trials=[]


for i, epoch_sd in enumerate(epochs_sd):    # for each epoch, channel
    # Boolean array where True indicates a channel's sd exceeds the threshold 
    bad_channels_mask = (epoch_sd > 3*mean_time_sd)|  (epoch_sd <- 3*mean_time_sd)     #or epoch_sd < mean_sd - 3 * std_sd  #mean_sd mean_sd+ 2
    #print(bad_channels_mask)
    if np.any(bad_channels_mask):
        bad_trials.append(i)
    # Get the names of the bad channels
    bad_channels = channel_names[bad_channels_mask]
    
    bad_channels_per_epoch[i] = bad_channels.tolist()

# # Drop bad trials
epochs_new.drop(bad_trials, reason='HIGH_STD')